- Pre process
- Make all sentance Equal length
- Make all articles of equal length
- Get BERT embeddings for sentances and words
- Make graph taking sentance as rows and words and label as column
- Feed in graph attention model for sentance classification

# Import Required Packages

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords


import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel


from IPython.display import clear_output # to clear the large outputs

In [2]:
# Initialize NLTK
nltk.download('punkt')
clear_output()

In [3]:
# df = pd.read_csv("../EnglishNews_train.csv", encoding="utf-8", nrows=10)
df = pd.read_csv("./newEnglishNews_train.csv", encoding="utf-8", nrows=1000).dropna().reset_index().drop(['index'], axis=1)
df.head()

,Heading,Summary,Article,id
0,"Barbie Review: Greta Gerwig, Margot Robbie, Ry...",Barbie Movie Review: Ryan Gosling shines the b...,Barbie Movie Review: One mention of Barbie and...,EnglishNews_train_0
1,Gadar 2: Sunny Deol-Ameesha Patel Starrer's Tr...,A source close to the film told News18 exclusi...,"The highly anticipated Gadar 2, starring Sunny...",EnglishNews_train_1
2,Kartik Aaryan Ditches First Class To Fly In Ec...,Kartik Aaryan was spotted flying in economy cl...,"Kartik Aaryan, who is gearing up for the relea...",EnglishNews_train_2
3,"India's Anju, Now Fatima, Receives Land, Money...",Abbasi had said that it was important to make ...,"Indian woman, Anju who travelled to Pakistan t...",EnglishNews_train_3
4,Himachal Pradesh Hotels Offer 50% Discount As ...,Flash floods and landslides caused by heavy do...,"Himachal Pradesh, one of the worst hit states ...",EnglishNews_train_4


In [4]:
df.shape

(970, 4)

# Preprocess and word tokenize all articles

In [5]:
articles = df["Article"]
article = articles[0]
article

'Barbie Movie Review: One mention of Barbie and you instantly think of a lean, tall-figured doll who is always well-dressed and has a perfect life. This stereotype is stuck with the doll for years, despite the numerous new designs and models being introduced. Director Greta Gerwig understood these stereotypes and even plays along with them for the first 15 to 20 minutes of the Barbie movie. However, she shatters the glass as soon as she is done playing with the &amp;#8216;doll&amp;#8217; and gets down to business to tell a relatable, feminist, and applaud-worthy movie that lives up to the hype.Barbie, starring Margot Robbie in the lead, is set in Barbie Land which has several kinds of Barbies co-existing. We see the stereotype Barbie (Margot), the President Barbie (Issa Rae), Physicist Barbie (Emma Mackey), and Lawyer Barbie (Sharon Rooney), among many more, running the Barbie land while the men, all of them named Ken, double up as their supporters, lovers, and partners.All goes well i

In [6]:
all_summary = df["Summary"]
summary = all_summary[0]
summary

'Barbie Movie Review: Ryan Gosling shines the brightest in the Greta Gerwig film. Margot Robbie proves she was meant for this role.'

In [7]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
clear_output()

# Preprocess the text
def preprocess(text):
    text = ' '.join(nltk.word_tokenize(text))
    
    # Lowercase
    text = text.lower()

    # Replace the newlines and punctuations with space
    filters = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    text = text.translate(str.maketrans(filters, ' '*len(filters)))

    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # Remove punctuations and numbers
    text = ' '.join([word for word in text.split() if word.isalpha()])
    
    # Remove single character
    text = ' '.join([word for word in text.split() if len(word) > 2])
    return text

In [8]:
def word_tokenize_articles(articles):
    preprocessed_articles = []
    word_tokenized_articles_list = []
    for article in articles:
        sentences = nltk.sent_tokenize(article)
        preprocessed_sentences = [preprocess(sentence) for sentence in sentences]

        # Word tokenization after preprocessing
        word_tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in preprocessed_sentences]
        word_tokenized_articles_list.append(word_tokenized_sentences)
        preprocessed_articles.append(preprocessed_sentences)
    
    return word_tokenized_articles_list

In [9]:
word_tokenized_articles = word_tokenize_articles(articles)
print("Preprocessed and word tokenized article: ", word_tokenized_articles[0])

Preprocessed and word tokenized article:  [['barbie', 'movie', 'review', 'one', 'mention', 'barbie', 'instantly', 'think', 'lean', 'tall', 'figured', 'doll', 'always', 'well', 'dressed', 'perfect', 'life'], ['stereotype', 'stuck', 'doll', 'years', 'despite', 'numerous', 'new', 'designs', 'models', 'introduced'], ['director', 'greta', 'gerwig', 'understood', 'stereotypes', 'even', 'plays', 'along', 'first', 'minutes', 'barbie', 'movie'], ['however', 'shatters', 'glass', 'soon', 'done', 'playing', 'amp', 'doll', 'amp', 'gets', 'business', 'tell', 'relatable', 'feminist', 'applaud', 'worthy', 'movie', 'lives', 'hype', 'barbie', 'starring', 'margot', 'robbie', 'lead', 'set', 'barbie', 'land', 'several', 'kinds', 'barbies', 'existing'], ['see', 'stereotype', 'barbie', 'margot', 'president', 'barbie', 'issa', 'rae', 'physicist', 'barbie', 'emma', 'mackey', 'lawyer', 'barbie', 'sharon', 'rooney', 'among', 'many', 'running', 'barbie', 'land', 'men', 'named', 'ken', 'double', 'supporters', 'lov

# Generate BERT embeddings for setances and words

## Make TF-IDF functions

In [10]:
# Get IDF from here and make tf for each sentence while making bert embeddings
def get_idf(word_tokenized_article):
    words_frequency = {}                    # Number of times a word appears in the document
    total_sentences_containing_word = {}    # Number of sentences containing a word
    words_idf = {}                          # IDF of each word

    for sentence in word_tokenized_article:
        for word in sentence:
            if word not in words_frequency.keys():
                words_frequency[word] = 1
            else:
                words_frequency[word] += 1
        
        for word in set(sentence):
            if word not in total_sentences_containing_word.keys():
                total_sentences_containing_word[word] = 1
            else:
                total_sentences_containing_word[word] += 1


    for word in words_frequency.keys():
        words_idf[word] = np.log(len(word_tokenized_article) / words_frequency[word])

    return words_frequency, words_idf


## Generate sentance and word embedding for each article

In [11]:
# Define a BERT model and tokenizer (replace with the specific BERT model you are using)
model_name = "bert-base-uncased"  # Example: You can use a different pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)
clear_output()

In [12]:
def get_embeddings(word_tokenize_article):
    # Initialize lists to store sentence and word embeddings
    sentence_embeddings = []
    word_embeddings = []

    # Store the tokenized input IDs, attention masks and token type IDs
    input_ids = []
    attention_masks = []


    for sentence in word_tokenize_article:
        # Convert words to BERT tokens
        tokens = [tokenizer.cls_token] + sentence + [tokenizer.sep_token]

        # Convert tokens to input IDs
        _input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Create attention masks by tf-idf values (freq of word in sentence / total sentences containing word)
        words_frequency, words_idf = get_idf(word_tokenize_article)

        words_tf = {}
        for word in sentence:
            if word not in words_tf.keys():
                words_tf[word] = 1
            else:
                words_tf[word] += 1
        
        attention_mask = [0] + [words_tf[word]/words_frequency[word] * words_idf[word] for word in sentence] + [0]


        # Create an input dictionary in the expected format
        input_dict = {
            'input_ids': tf.constant([_input_ids]),
            'attention_mask': tf.constant([attention_mask]),
        }

        # Get BERT model output
        with tf.device('/GPU:0'):
            output = model(input_dict)

        # Extract sentence and word embeddings
        sentence_embedding = tf.reduce_mean(output.last_hidden_state, axis=1).numpy()  # Sentence embedding
        word_embedding = output.last_hidden_state.numpy()  # Word embeddings

        # Append to lists
        sentence_embeddings.append(sentence_embedding.reshape(768, ))
        word_embeddings.append(word_embedding.reshape(-1, 768))

        # Append to lists Attention masks and input IDs
        input_ids.append(tf.constant([_input_ids]).numpy().reshape(-1))
        attention_masks.append(tf.constant([attention_mask]).numpy().reshape(-1))

    return sentence_embeddings, word_embeddings

In [13]:
# Store the sentence and word embeddings for each article 
all_articles_sentence_embeddings = []
all_articles_word_embeddings = []

for article in word_tokenized_articles[:5]:
    sentence_embedding, word_embedding = get_embeddings(article)
    all_articles_sentence_embeddings.append(sentence_embedding)
    all_articles_word_embeddings.append(word_embedding)


# Make all articles of equal length

In [14]:
max_article_len = 0

for article in word_tokenized_articles:
    max_article_len = max(max_article_len, len(article))

In [15]:
padded_articles = []

# Padding and truncating the articles
for article in word_tokenized_articles:
    while len(article) < max_article_len:
        article.append([])
    while len(article) > max_article_len:
        article.pop()
    padded_articles.append(article)

print("Padded and truncated article: ", padded_articles[0])

Padded and truncated article:  [['barbie', 'movie', 'review', 'one', 'mention', 'barbie', 'instantly', 'think', 'lean', 'tall', 'figured', 'doll', 'always', 'well', 'dressed', 'perfect', 'life'], ['stereotype', 'stuck', 'doll', 'years', 'despite', 'numerous', 'new', 'designs', 'models', 'introduced'], ['director', 'greta', 'gerwig', 'understood', 'stereotypes', 'even', 'plays', 'along', 'first', 'minutes', 'barbie', 'movie'], ['however', 'shatters', 'glass', 'soon', 'done', 'playing', 'amp', 'doll', 'amp', 'gets', 'business', 'tell', 'relatable', 'feminist', 'applaud', 'worthy', 'movie', 'lives', 'hype', 'barbie', 'starring', 'margot', 'robbie', 'lead', 'set', 'barbie', 'land', 'several', 'kinds', 'barbies', 'existing'], ['see', 'stereotype', 'barbie', 'margot', 'president', 'barbie', 'issa', 'rae', 'physicist', 'barbie', 'emma', 'mackey', 'lawyer', 'barbie', 'sharon', 'rooney', 'among', 'many', 'running', 'barbie', 'land', 'men', 'named', 'ken', 'double', 'supporters', 'lovers', 'part

# Create a adjecency matrix for each article


For each article need to make a s*w matrix

- w:= Unique words in the whole dataset
- s:- Number of sentances in each article